In [39]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import numpy as np
import pandas as pd
from nltk.stem import WordNetLemmatizer
from string import digits
from nltk.corpus import stopwords
import string
import nltk
from sklearn.metrics import confusion_matrix
from random import randint
import numpy as np
import torch
import sklearn
import operator
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [0]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import os, sys

In [0]:
url1 = "https://raw.githubusercontent.com/varun-spectre/assignment/master/train.csv"
url2 = "https://raw.githubusercontent.com/varun-spectre/assignment/master/test.csv"

In [0]:
"""
Not removing digits as date-time may also lead to a pattern as per problem statement
By analysing the data, Key words like "Please" "Could" should not be removed as they are also leading to a pattern.
So here I am just removing stopwords which has length <=3
"""
lemmatizer = WordNetLemmatizer()
def preProcess(sentence):
    stop = set(stopwords.words('english'))
    remove_punctuation = str.maketrans('','',string.punctuation)
#     remove_digits = str.maketrans('','',digits)
    sentence = sentence.translate(remove_punctuation)
#     sentence = sentence.translate(remove_digits).translate(remove_punctuation)
    tokens = [i for i in sentence.lower().split() if (i not in stop or len(i) > 3)]
    s=""
    for token in tokens:
        token = lemmatizer.lemmatize(token)
        s = s + ' ' + token
    return s;

In [0]:
df = pd.read_csv(url1)

In [0]:
df["NewText"] = ""
for i in range(df.shape[0]):
    df["NewText"][i] = preProcess(df["Text"][i])
df['Tag'] = df['Label'].eq('Yes').astype(int)  

In [0]:
df1 = pd.read_csv(url2)
df1["NewText"] = ""
for i in range(df1.shape[0]):
    df1["NewText"][i] = preProcess(df1["Text"][i])
df1['Tag'] = df1['Label'].eq('Yes').astype(int) 

In [0]:
list1  = df["NewText"].tolist()
list2  = df1["NewText"].tolist() 
for i in range(len(list2)):
    list1.append(list2[i])

In [27]:
np.shape(list1)

(4649,)

In [0]:
module_url = "https://tfhub.dev/google/universal-sentence-encoder/2"

In [0]:
embed = hub.Module(module_url)

In [0]:
tf.logging.set_verbosity(tf.logging.ERROR)
with tf.Session() as session:
  session.run([tf.global_variables_initializer(), tf.tables_initializer()])
  sentences_embeddings = session.run(embed(list1))


In [0]:
X = sentences_embeddings

In [0]:
X_train = X[:3657]
X_test = X[3657:]
y_train = df['Tag'].tolist()
y_test = df1['Tag'].tolist()
y_train = np.asarray(y_train)
y_test = np.asarray(y_test)
y_train = np.reshape(y_train,(3657,1))
y_test = np.reshape(y_test,(992,1))

In [35]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(3657, 512)
(992, 512)
(3657, 1)
(992, 1)


In [0]:
from xgboost import XGBClassifier

In [37]:
model = XGBClassifier()
model.fit(X_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [40]:
y_pred = model.predict(X_test)
accuracy_score(y_test,y_pred)

0.751008064516129

In [0]:
X_train = X[:3657]
X_test = X[3657:]
y_train = df['Tag'].tolist()
y_test = df1['Tag'].tolist()
y_train = np.asarray(y_train)
y_test = np.asarray(y_test)
y_train = np.reshape(y_train,(3657,1))
y_test = np.reshape(y_test,(992,1))

In [42]:
from sklearn.svm import SVC
clf = SVC(C=2300, kernel='rbf')
clf.fit(X_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVC(C=2300, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='rbf', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

In [0]:
y_pred = clf.predict(X_test)

In [44]:
accuracy_score(y_test,y_pred)

0.7429435483870968